# Retail Banking Challenge 1 - Baseline Submission

This notebook provides a simple baseline for **Retail Banking Challenge 1: Fraud Detection**.

**Goal**: Predict `FraudLabel` (0/1) for each transaction
**Metric**: Macro-F1 - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular transaction data with a simple Random Forest classifier.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier



# Load data from PVC paths
print("📂 Loading Retail Banking Challenge 1 data...")

# Load transaction data
train_txns = pd.read_csv("C:/Users/Feifei/Downloads/Rb/transactions_train.csv")
test_txns = pd.read_csv("C:/Users/Feifei/Downloads/Rb/transactions_test.csv")

print(f"✅ Data loaded:")
print(f"   Train transactions: {train_txns.shape}")
print(f"   Test transactions: {test_txns.shape}")
print(f"   Train columns: {list(train_txns.columns)}")
print(f"   Test columns: {list(test_txns.columns)}")


In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# From data inspection - transaction columns:
# Timestamp_dt, TxnID, Timestamp, SessionID, CustomerID, SrcAccount, DstAccount, Channel, MCC_Group, Amount, FraudLabel (train only)

# Select features for baseline (exclude ID and target columns)
txn_features = ['Amount', 'Channel', 'MCC_Group']
print(f"📊 Using transaction features: {txn_features}")

# Prepare training data
X_train = train_txns[txn_features].fillna(0)
# Encode categorical features
for col in ['Channel', 'MCC_Group']:
    if col in X_train.columns:
        X_train[col] = pd.Categorical(X_train[col]).codes

X_train = X_train.astype(float)
y_train = train_txns['FraudLabel'].astype(int)  # Target variable

# Prepare test data
X_test = test_txns[txn_features].fillna(0)
# Encode categorical features (use same encoding as training)
for col in ['Channel', 'MCC_Group']:
    if col in X_test.columns:
        X_test[col] = pd.Categorical(X_test[col]).codes

X_test = X_test.astype(float)

# Train simple Random Forest classifier baseline
print("🤖 Training Random Forest classifier...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Create submission file (format: TxnID,FraudLabel)
submission_df = pd.DataFrame({
    'TxnID': test_txns['TxnID'],
    'FraudLabel': predictions
})

# Save predictions
submission_df.to_csv("retailbanking_challenge1_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Fraud rate: {predictions.mean():.3f} ({predictions.sum()} fraud cases out of {len(predictions)})")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Retailbanking", 1, "retailbanking_challenge1_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Retail Banking Challenge 2!")
